#  Credit Risk WoE Binning

### This notebook performs Weight of Evidence (WoE) binning for RFM features ,goal is to preprocess features for a credit risk model.

In [1]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))
from credit_risk_woe import calculate_woe, visualize_woe, setup_logging

